# Logistic Regression Implementation from scratch

In [211]:
#Getting all the important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

### Exploring our dataset

We have a total of 11 input columns and 1 output column which represents whether the person has a Cardiovascular disease or not.
We have roughly 150 entries for each classification so it is a balanced dataset. No more processing is required on it.

In [212]:
#loading the dataset
heart = pd.read_csv("heart.csv")

#shuffling the dataset 
heart = heart.sample(frac=1).reset_index(drop=True)

#Seperating the input and output variables
X = heart[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
Y = heart["target"]

#Splitting the dataset in to test and training samples
sample_x = X[:250]
sample_y = Y[:250].values
sample_y = sample_y.reshape(sample_y.shape[0],-1)
test_x = X[250:]
test_y = Y[250:].values
test_y = test_y.reshape(test_y.shape[0],-1)

In [213]:
sample_x.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,34,1,3,118,182,0,0,174,0,0.0,2,0,2
1,42,0,0,102,265,0,0,122,0,0.6,1,0,2
2,62,0,0,150,244,0,1,154,1,1.4,1,0,2
3,56,1,3,120,193,0,0,162,0,1.9,1,0,3
4,50,1,0,150,243,0,0,128,0,2.6,1,0,3


In [214]:
print(sample_x.shape)
print(sample_y.shape)

(250, 13)
(250, 1)


Defining helper functions for the aglorithm

In [215]:
def get_sigmoid(inp):
    
    #simply returning sigmoid our values
    return (1/(1+np.exp(-inp))) - 0.00000001
    

In [216]:
def get_loss(y_hat, y, n):
    
    #simply returning the loss calculated using our loss function
    return -(1/n)*(np.sum(y*(np.log(y_hat)) + (1-y)*np.log(1-y_hat)))
    

In [217]:
def update_params(n, X, y_hat, y, alpha, w_old):
  
    #looping to update every parameter
    diff = y_hat-y
    update = (1/n)*np.dot(X.T,diff)
   
    #applying the update equation
    w_new = w_old-(alpha)*(update.T)
    
    #return the updated weights
    return w_new

In [218]:
def train_logistic_regression(X,y, alpha = 0.01, epochs = 10000):
    
    #first we get the number of examples present in our dataset
    n = len(X)
    
    #normalise data using min_max scaling
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #Randomly initialising weights of our model
    w = np.random.rand(1,X.shape[1])
    print("Original weights =",w)
    
    #setting an initial value for loss (a high value)
    #loss = 1000
    for i in range(epochs):
        
        dot = np.dot(X, w.T)
        y_hat = get_sigmoid(dot)
        
        #calculate the loss against the predicted outputs
        loss = get_loss(y_hat, y, n)
        
        print("Loss ================>",loss)
        
        #updating the weights based on the loss
        w = update_params(n, X, y_hat, y, alpha, w)
    
    #we return the predictions made on the train dataset as well as the final weights.
    return w, y_hat

Lets train our built model

In [219]:
%%time
weights, out = train_logistic_regression(sample_x,sample_y)

Original weights = [[0.31129175 0.51786169 0.43548714 0.13574612 0.41867452 0.86717353
  0.25890865 0.44446465 0.86472381 0.75065537 0.62682723 0.67233518
  0.48020475]]
Loss ================> 1.318230417668562
Loss ================> 1.314199591418732
Loss ================> 1.3101814357781332
Loss ================> 1.3061760138327034
Loss ================> 1.3021833883250047
Loss ================> 1.2982036216420294
Loss ================> 1.294236775802954
Loss ================> 1.290282912446846
Loss ================> 1.2863420928203324
Loss ================> 1.2824143777652308
Loss ================> 1.2784998277061448
Loss ================> 1.2745985026380344
Loss ================> 1.2707104621137593
Loss ================> 1.2668357652316047
Loss ================> 1.26297447062279
Loss ================> 1.2591266364389668
Loss ================> 1.2552923203397153
Loss ================> 1.2514715794800308
Loss ================> 1.2476644704978237
Loss ================> 1.2438710495014

Loss ================> 0.5241922175395296
Loss ================> 0.5241076706189662
Loss ================> 0.524023202995326
Loss ================> 0.5239388145669814
Loss ================> 0.5238545052324578
Loss ================> 0.5237702748904317
Loss ================> 0.5236861234397312
Loss ================> 0.5236020507793363
Loss ================> 0.5235180568083775
Loss ================> 0.5234341414261363
Loss ================> 0.5233503045320447
Loss ================> 0.5232665460256855
Loss ================> 0.5231828658067909
Loss ================> 0.5230992637752434
Loss ================> 0.523015739831075
Loss ================> 0.5229322938744672
Loss ================> 0.5228489258057506
Loss ================> 0.5227656355254048
Loss ================> 0.5226824229340578
Loss ================> 0.522599287932486
Loss ================> 0.5225162304216144
Loss ================> 0.5224332503025159
Loss ================> 0.5223503474764106
Loss ================> 0.522267521844

Loss ================> 0.45825671984484884
Loss ================> 0.45822408838208234
Loss ================> 0.4581914769017986
Loss ================> 0.45815888538543015
Loss ================> 0.45812631381443153
Loss ================> 0.4580937621702797
Loss ================> 0.4580612304344737
Loss ================> 0.45802871858853483
Loss ================> 0.45799622661400663
Loss ================> 0.4579637544924546
Loss ================> 0.4579313022054661
Loss ================> 0.45789886973465094
Loss ================> 0.45786645706164086
Loss ================> 0.4578340641680894
Loss ================> 0.4578016910356722
Loss ================> 0.4577693376460868
Loss ================> 0.45773700398105244
Loss ================> 0.4577046900223107
Loss ================> 0.45767239575162444
Loss ================> 0.4576401211507787
Loss ================> 0.45760786620157995
Loss ================> 0.4575756308858568
Loss ================> 0.45754341518545916
Loss ================>

Loss ================> 0.4257969655342478
Loss ================> 0.42578100840538036
Loss ================> 0.4257650578904488
Loss ================> 0.4257491139851832
Loss ================> 0.42573317668531796
Loss ================> 0.4257172459865911
Loss ================> 0.425701321884744
Loss ================> 0.4256854043755224
Loss ================> 0.42566949345467514
Loss ================> 0.42565358911795553
Loss ================> 0.42563769136111995
Loss ================> 0.42562180017992896
Loss ================> 0.4256059155701465
Loss ================> 0.42559003752754054
Loss ================> 0.4255741660478826
Loss ================> 0.425558301126948
Loss ================> 0.42554244276051567
Loss ================> 0.4255265909443684
Loss ================> 0.4255107456742924
Loss ================> 0.425494906946078
Loss ================> 0.42547907475551894
Loss ================> 0.42546324909841265
Loss ================> 0.42544742997056034
Loss ================> 0.4

Loss ================> 0.409176418868078
Loss ================> 0.40916654430187227
Loss ================> 0.4091566728857405
Loss ================> 0.4091468046181497
Loss ================> 0.4091369394975675
Loss ================> 0.4091270775224632
Loss ================> 0.40911721869130646
Loss ================> 0.40910736300256845
Loss ================> 0.4090975104547211
Loss ================> 0.40908766104623756
Loss ================> 0.40907781477559185
Loss ================> 0.40906797164125924
Loss ================> 0.40905813164171584
Loss ================> 0.40904829477543875
Loss ================> 0.4090384610409064
Loss ================> 0.4090286304365978
Loss ================> 0.4090188029609935
Loss ================> 0.4090089786125747
Loss ================> 0.4089991573898236
Loss ================> 0.4089893392912238
Loss ================> 0.4089795243152594
Loss ================> 0.4089697124604161
Loss ================> 0.4089599037251802
Loss ================> 0.40

Loss ================> 0.3972251504834692
Loss ================> 0.3972187150695284
Loss ================> 0.39721228131342673
Loss ================> 0.3972058492145361
Loss ================> 0.39719941877222875
Loss ================> 0.3971929899858774
Loss ================> 0.3971865628548547
Loss ================> 0.3971801373785341
Loss ================> 0.3971737135562891
Loss ================> 0.39716729138749357
Loss ================> 0.3971608708715218
Loss ================> 0.39715445200774824
Loss ================> 0.39714803479554783
Loss ================> 0.39714161923429575
Loss ================> 0.3971352053233675
Loss ================> 0.39712879306213894
Loss ================> 0.3971223824499863
Loss ================> 0.39711597348628613
Loss ================> 0.3971095661704151
Loss ================> 0.39710316050175054
Loss ================> 0.3970967564796698
Loss ================> 0.3970903541035508
Loss ================> 0.3970839533727717
Loss ================> 0.

Loss ================> 0.38952757706941143
Loss ================> 0.38952298417167774
Loss ================> 0.389518392291182
Loss ================> 0.3895138014276031
Loss ================> 0.3895092115806198
Loss ================> 0.38950462274991104
Loss ================> 0.3895000349351562
Loss ================> 0.38949544813603426
Loss ================> 0.3894908623522249
Loss ================> 0.3894862775834075
Loss ================> 0.38948169382926173
Loss ================> 0.3894771110894675
Loss ================> 0.3894725293637047
Loss ================> 0.38946794865165346
Loss ================> 0.38946336895299405
Loss ================> 0.3894587902674067
Loss ================> 0.389454212594572
Loss ================> 0.3894496359341707
Loss ================> 0.3894450602858835
Loss ================> 0.38944048564939127
Loss ================> 0.38943591202437505
Loss ================> 0.3894313394105161
Loss ================> 0.3894267678074958
Loss ================> 0.38

Loss ================> 0.38411774351201644
Loss ================> 0.3841142829482919
Loss ================> 0.3841108230670939
Loss ================> 0.38410736386823435
Loss ================> 0.384103905351526
Loss ================> 0.38410044751678113
Loss ================> 0.38409699036381223
Loss ================> 0.384093533892432
Loss ================> 0.38409007810245305
Loss ================> 0.38408662299368806
Loss ================> 0.38408316856594993
Loss ================> 0.38407971481905145
Loss ================> 0.3840762617528056
Loss ================> 0.38407280936702537
Loss ================> 0.3840693576615238
Loss ================> 0.38406590663611406
Loss ================> 0.3840624562906093
Loss ================> 0.38405900662482295
Loss ================> 0.3840555576385681
Loss ================> 0.38405210933165834
Loss ================> 0.3840486617039072
Loss ================> 0.38404521475512815
Loss ================> 0.38404176848513477
Loss ================>

Since our dataset was a relatively small one so it only took 1.35 s to train with 1000 epochs

In [220]:
weights

array([[-0.63954008, -1.07365887,  1.98337025, -0.46291947, -0.09634861,
         0.19209945,  0.79558596,  1.598035  , -0.86770595, -0.71620467,
         1.81328009, -2.03994124, -1.02228925]])

In [221]:
def predict_logistic_regression(weights, x_test):
    
    #doing the neccesary calculations before applying sigmoid funciton
    X = x_test.values
    #normalise data using min_max scaling
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #calculate the predictions
    product = np.dot(X,weights.T)
    y_preds = get_sigmoid(product)
    
    #return the predictions with applied thresholds
    return [1 if i >= 0.5 else 0 for i in y_preds]

In [222]:
y_pred = predict_logistic_regression(weights, test_x)

In [223]:
'''
funciton to calculate the percentage accuracy from the given predictions
'''

def get_accuracy(y_pred, y):
    
    assert len(y_pred) == len(y)
    n = len(y)
    Y = y
    
    count = 0
    for i in range(n):
        if y_pred[i] == Y[i]:
            count+=1
     
    return print("Accuracy:", round(((count/n)*100),2), "%")

In [224]:
get_accuracy(y_pred, test_y)

Accuracy: 75.47 %


An accuracy of **75.47%** is reasonable, we can increase the accuracy by training the using different hyperparameters *(learning_rate & epochs)*.

Lets see how well we have performed compared to the built-in function from Sk-Learn library



# Logistic Regression from SK-Learn

In [233]:
#loading new dataset
heart = pd.read_csv("heart.csv")
heart = heart.sample(frac=1).reset_index(drop=True)


In [234]:
X = heart[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
Y = heart["target"]
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [235]:
sample_x = X[:250]
sample_y = Y[:250].values
test_x = X[250:]
test_y = Y[250:].values

In [236]:
 from sklearn.linear_model import LogisticRegression

In [237]:
clf = LogisticRegression().fit(X, Y)

In [238]:
logit_preds = clf.predict(test_x)

In [239]:
get_accuracy(logit_preds, test_y)

Accuracy: 79.25 %


It yields an accuracy of **79.25%** hence our model has performed quite well in the first try.